In [1]:
# FINAL PROJECT DATA ENGINEERING (Short Trade Program Script)
# Author - Priya Kundu and Nikitaa Kenkre
# Date of Program Run - 11th May 2023 (3pm to 7:30am)
# Date of Presentation - 12th May 2023 (11am)
# Semester Spring 2023
# By Professor Carlos De Oliveira

# Importing Packages and Setting Parameters

In [2]:
import oandapyV20
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.pricing as pricing
from oandapyV20.contrib.requests import MarketOrderRequest
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
import pandas as pd
import datetime as dt
import time
from datetime import datetime

In [3]:
# OANDA account details
access_token = "681073222fb4cf28caa21c5d2bec7322-4a90092d303c32495bdc15a63a9708ee"
accountID = "101-001-25499063-005"
#api = oandapyV20.API(access_token=access_token, environment="practice")
try:
    client = oandapyV20.API(access_token=access_token, environment="practice")
except V20Error as e:
    print("Error: {}".format(e))

In [4]:
# specify the instrument to trade
instrument = "USD_CHF" 

In [5]:
# set the order parameters
order_quantity = 1000 # start with 1000 units
total_order_quantity = 100000 # the total order quantity
current_executed_quantity = 0 # keep track of how many units have been executed so far
non_executed_quantity = 0 # keep track of the units that were not executed in the previous window
average_execution_price = 0 # initialize the average execution price to 0
total_average_execution_price = 0 # initialize the total average execution price to 0

In [6]:
# setting the required time frames
first_execution_start_time = dt.datetime.now().replace(hour=15, minute=0, second=0, microsecond=0)
first_execution_end_time = dt.datetime.now().replace(hour=17, minute=0, second=0, microsecond=0)

second_execution_start_time = dt.datetime.now().replace(hour=19, minute=0, second=0, microsecond=0)
second_execution_end_time = dt.datetime.now().replace(hour=22, minute=0, second=0, microsecond=0)

third_execution_start_time = dt.datetime.now().replace(hour=23, minute=0, second=0, microsecond=0)
third_execution_end_time = dt.datetime.now().replace(hour=1, minute=0, second=0, microsecond=0) + dt.timedelta(days=1) # add one day to the end time

fourth_execution_start_time = dt.datetime.now().replace(hour=3, minute=0, second=0, microsecond=0) + dt.timedelta(days=1) # add one day to the start time
fourth_execution_end_time = dt.datetime.now().replace(hour=6, minute=0, second=0, microsecond=0) + dt.timedelta(days=1) # add one day to the end time

In [7]:
# define a function to execute orders
def execute_order(units, instrument, side, trades):
    # create a market order request
    direction = 1 if side == "BUY" else -1
    mo = MarketOrderRequest(
        instrument=instrument,
        units=str(int(units)*direction),
        takeProfitOnFill=None,
        stopLossOnFill=None,
    )
    # send the order request
    r = orders.OrderCreate(accountID, data=mo.data)
    response = client.request(r)
    if "orderFillTransaction" in response:
        trades.append({
            "Timestamp": dt.datetime.now(), 
            "Order_ID": float(response["orderFillTransaction"]["orderID"]),
            "Instrument": instrument,
            "Price": float(response["orderFillTransaction"]["price"]),
            "Units": int(units),
            "Side": side
        })
    else:
        # Handle the case where the order is not filled
        if "orderCancelTransaction" in response:
            print("Order canceled due to:", response["orderCancelTransaction"]["reason"])
        else:
            print("No cancel transaction found.")

In [8]:
# make a function to get price that can be used to compare with the average price before execution takes place
def get_rate():
    price_request = pricing.PricingInfo(accountID=accountID, params={'instruments': instrument})
    response = client.request(price_request)
    return float(response['prices'][0]['asks'][0]['price'])

# First Window

In [9]:
# execute the first window
while dt.datetime.now() < first_execution_start_time:
    time.sleep(1)

batch1_trades = []  # create a list to store the prices

while dt.datetime.now() >= first_execution_start_time and dt.datetime.now() <= first_execution_end_time:
    for i in range(20): # 20 iterations to execute 1000 units every 6 minutes
        if current_executed_quantity < total_order_quantity * 0.2:       
            response = execute_order(order_quantity, instrument, "SELL", batch1_trades)
            current_executed_quantity += order_quantity
        else:
            break
        time.sleep(360) # wait for 6 minutes between orders
        print("Successful execution for", i, "th interval") 

Successful execution for 0 th interval
Successful execution for 1 th interval
Successful execution for 2 th interval
Successful execution for 3 th interval
Successful execution for 4 th interval
Successful execution for 5 th interval
Successful execution for 6 th interval
Successful execution for 7 th interval
Successful execution for 8 th interval
Successful execution for 9 th interval
Successful execution for 10 th interval
Successful execution for 11 th interval
Successful execution for 12 th interval
Successful execution for 13 th interval
Successful execution for 14 th interval
Successful execution for 15 th interval
Successful execution for 16 th interval
Successful execution for 17 th interval
Successful execution for 18 th interval
Successful execution for 19 th interval


In [10]:
# convert the list of prices into a DataFrame
df1 = pd.DataFrame(batch1_trades)

df1

,Timestamp,Order_ID,Instrument,Price,Units,Side
0,2023-05-11 15:00:01.141069,20.0,USD_CHF,0.89433,1000,SELL
1,2023-05-11 15:06:01.229906,24.0,USD_CHF,0.89443,1000,SELL
2,2023-05-11 15:12:01.337653,30.0,USD_CHF,0.89409,1000,SELL
3,2023-05-11 15:18:01.444667,38.0,USD_CHF,0.89409,1000,SELL
4,2023-05-11 15:24:01.544418,42.0,USD_CHF,0.89408,1000,SELL
5,2023-05-11 15:30:01.638780,48.0,USD_CHF,0.89399,1000,SELL
6,2023-05-11 15:36:01.759227,54.0,USD_CHF,0.89376,1000,SELL
7,2023-05-11 15:42:01.830216,60.0,USD_CHF,0.89393,1000,SELL
8,2023-05-11 15:48:01.917581,66.0,USD_CHF,0.89408,1000,SELL
9,2023-05-11 15:54:02.046035,74.0,USD_CHF,0.89395,1000,SELL


In [11]:
average_execution_price1 = df1['Price'].sum() / len(df1)

average_execution_price1

0.8940465000000002

In [12]:
current_executed_quantity

20000

# Second Window

In [13]:
# execute the second window
while dt.datetime.now() < second_execution_start_time:
    time.sleep(1)

batch2_trades = []  # create a list to store the prices

while dt.datetime.now() >= second_execution_start_time and dt.datetime.now() <= second_execution_end_time:
    for i in range(30): # 30 iterations to execute 1000 units every 6 minutes
        if current_executed_quantity < total_order_quantity * 0.5:
            if get_rate() <= average_execution_price1:
                execute_order(order_quantity, instrument, "SELL", batch2_trades)
                current_executed_quantity += order_quantity
                print("Trade took place!")
            else:
                print("Trade did not take place as Price < Average Price")
        else:
            break
        time.sleep(360) # wait for 6 minutes between orders
        print("Successful execution for", i, "th interval")

Trade did not take place as Price < Average Price
Successful execution for 0 th interval
Trade did not take place as Price < Average Price
Successful execution for 1 th interval
Trade did not take place as Price < Average Price
Successful execution for 2 th interval
Trade did not take place as Price < Average Price
Successful execution for 3 th interval
Trade did not take place as Price < Average Price
Successful execution for 4 th interval
Trade did not take place as Price < Average Price
Successful execution for 5 th interval
Trade did not take place as Price < Average Price
Successful execution for 6 th interval
Trade did not take place as Price < Average Price
Successful execution for 7 th interval
Trade did not take place as Price < Average Price
Successful execution for 8 th interval
Trade did not take place as Price < Average Price
Successful execution for 9 th interval
Trade did not take place as Price < Average Price
Successful execution for 10 th interval
Trade did not take p

In [14]:
# convert the list of prices into a DataFrame
df2 = pd.DataFrame(batch2_trades)

df2

,Timestamp,Order_ID,Instrument,Price,Units,Side
0,2023-05-11 20:24:52.834620,166.0,USD_CHF,0.89378,1000,SELL
1,2023-05-11 20:30:52.977250,169.0,USD_CHF,0.89375,1000,SELL
2,2023-05-11 20:36:53.122061,172.0,USD_CHF,0.89362,1000,SELL
3,2023-05-11 20:42:53.280198,175.0,USD_CHF,0.89373,1000,SELL
4,2023-05-11 20:48:53.529378,178.0,USD_CHF,0.89362,1000,SELL
5,2023-05-11 20:54:53.673634,181.0,USD_CHF,0.89378,1000,SELL
6,2023-05-11 21:00:53.838821,184.0,USD_CHF,0.89336,1000,SELL
7,2023-05-11 21:06:53.941399,190.0,USD_CHF,0.89350,1000,SELL
8,2023-05-11 21:12:54.105022,196.0,USD_CHF,0.89354,1000,SELL
9,2023-05-11 21:18:54.294158,202.0,USD_CHF,0.89357,1000,SELL


In [15]:
# concatenate the two dataframes vertically
new_df = pd.concat([df1, df2], axis=0)

In [16]:
# calculate the total average execution price
average_execution_price2 = new_df['Price'].sum() / len(new_df)

average_execution_price2

0.8938025

In [17]:
current_executed_quantity

36000

In [18]:
non_executed_quantity2 = total_order_quantity * 0.5 - current_executed_quantity

non_executed_quantity2

14000.0

# Third Window

In [20]:
# execute the third window
while dt.datetime.now() < third_execution_start_time:
    time.sleep(1)

batch3_trades = []  # create a list to store the prices

while dt.datetime.now() >= third_execution_start_time and dt.datetime.now() <= third_execution_end_time:
    # recalculate the number of units to be executed
    order_quantity = int((non_executed_quantity2 + (total_order_quantity * 0.2)) / 20)
    for i in range(20): # 20 iterations to execute every 6 minutes
        if current_executed_quantity < total_order_quantity * 0.7:
            if get_rate() <= average_execution_price2:
                response = execute_order(order_quantity, instrument, "SELL", batch3_trades)
                current_executed_quantity += order_quantity
                print("Trade took place!")
            else:
                print("Trade did not take place as Price < Average Price")
        else:
            break
        time.sleep(360) # wait for 6 minutes between orders
        print("Successful execution for", i, "th interval")

Trade took place!
Successful execution for 0 th interval
Trade took place!
Successful execution for 1 th interval
Trade took place!
Successful execution for 2 th interval
Trade took place!
Successful execution for 3 th interval
Trade took place!
Successful execution for 4 th interval
Trade took place!
Successful execution for 5 th interval
Trade took place!
Successful execution for 6 th interval
Trade took place!
Successful execution for 7 th interval
Trade took place!
Successful execution for 8 th interval
Trade took place!
Successful execution for 9 th interval
Trade took place!
Successful execution for 10 th interval
Trade took place!
Successful execution for 11 th interval
Trade took place!
Successful execution for 12 th interval
Trade took place!
Successful execution for 13 th interval
Trade took place!
Successful execution for 14 th interval
Trade took place!
Successful execution for 15 th interval
Trade took place!
Successful execution for 16 th interval
Trade took place!
Succes

In [21]:
# convert the list of prices into a DataFrame
df3 = pd.DataFrame(batch3_trades)

df3

,Timestamp,Order_ID,Instrument,Price,Units,Side
0,2023-05-11 23:00:00.872079,243.0,USD_CHF,0.89263,1700,SELL
1,2023-05-11 23:06:01.060024,249.0,USD_CHF,0.89272,1700,SELL
2,2023-05-11 23:12:01.306005,253.0,USD_CHF,0.89258,1700,SELL
3,2023-05-11 23:18:01.514459,256.0,USD_CHF,0.89273,1700,SELL
4,2023-05-11 23:24:01.687707,259.0,USD_CHF,0.89269,1700,SELL
5,2023-05-11 23:30:01.899599,262.0,USD_CHF,0.89268,1700,SELL
6,2023-05-11 23:36:02.069548,265.0,USD_CHF,0.89291,1700,SELL
7,2023-05-11 23:42:02.389705,268.0,USD_CHF,0.89293,1700,SELL
8,2023-05-11 23:48:02.579216,271.0,USD_CHF,0.89289,1700,SELL
9,2023-05-11 23:54:02.757775,274.0,USD_CHF,0.89276,1700,SELL


In [22]:
# concatenate the two dataframes vertically
updated_df = pd.concat([new_df, df3], axis=0)

In [23]:
# calculate the total average execution price
average_execution_price3 = updated_df['Price'].sum() / len(updated_df)

average_execution_price3

0.8934060714285714

In [24]:
current_executed_quantity

70000

In [25]:
non_executed_quantity3 = total_order_quantity * 0.7 - current_executed_quantity

non_executed_quantity3

0.0

# Fourth Window

In [26]:
# execute the fourth window
while dt.datetime.now() < fourth_execution_start_time:
    time.sleep(1)

batch4_trades = []  # create a list to store the prices

while dt.datetime.now() >= fourth_execution_start_time and dt.datetime.now() <= fourth_execution_end_time:
    # recalculate the number of units to be executed
    order_quantity = int((non_executed_quantity3 + (total_order_quantity * 0.3)) / 30)
    for i in range(30): # 30 iterations to execute every 6 minutes
        if current_executed_quantity < total_order_quantity:
            if get_rate() <= average_execution_price3:
                response = execute_order(order_quantity, instrument, "SELL", batch4_trades)
                current_executed_quantity += order_quantity
                print("Trade took place!")
            else:
                print("Trade did not take place as Price > Average Price")
        else:
            break
        time.sleep(360) # wait for 6 minutes between orders
        print("Successful execution for", i, "th interval")

Trade took place!
Successful execution for 0 th interval
Trade took place!
Successful execution for 1 th interval
Trade took place!
Successful execution for 2 th interval
Trade took place!
Successful execution for 3 th interval
Trade took place!
Successful execution for 4 th interval
Trade took place!
Successful execution for 5 th interval
Trade took place!
Successful execution for 6 th interval
Trade took place!
Successful execution for 7 th interval
Trade took place!
Successful execution for 8 th interval
Trade took place!
Successful execution for 9 th interval
Trade took place!
Successful execution for 10 th interval
Trade took place!
Successful execution for 11 th interval
Trade took place!
Successful execution for 12 th interval
Trade took place!
Successful execution for 13 th interval
Trade took place!
Successful execution for 14 th interval
Trade took place!
Successful execution for 15 th interval
Trade took place!
Successful execution for 16 th interval
Trade took place!
Succes

In [27]:
# convert the list of prices into a DataFrame
df4 = pd.DataFrame(batch4_trades)

df4

,Timestamp,Order_ID,Instrument,Price,Units,Side
0,2023-05-12 03:00:03.459445,307.0,USD_CHF,0.89197,1000,SELL
1,2023-05-12 03:06:03.662609,310.0,USD_CHF,0.89153,1000,SELL
2,2023-05-12 03:12:03.872172,313.0,USD_CHF,0.89135,1000,SELL
3,2023-05-12 03:18:04.127220,316.0,USD_CHF,0.89029,1000,SELL
4,2023-05-12 03:24:04.337856,319.0,USD_CHF,0.89001,1000,SELL
5,2023-05-12 03:30:04.555274,322.0,USD_CHF,0.89054,1000,SELL
6,2023-05-12 03:36:04.758546,325.0,USD_CHF,0.89109,1000,SELL
7,2023-05-12 03:42:04.927751,328.0,USD_CHF,0.89097,1000,SELL
8,2023-05-12 03:48:05.199417,331.0,USD_CHF,0.89105,1000,SELL
9,2023-05-12 03:54:05.368412,334.0,USD_CHF,0.89149,1000,SELL


In [28]:
# concatenate the two dataframes vertically
final_df = pd.concat([updated_df, df4], axis=0)

In [29]:
# calculate the total average execution price
final_average_execution_price = updated_df['Price'].sum() / len(updated_df)

final_average_execution_price

0.8934060714285714

In [30]:
current_executed_quantity

97000

In [31]:
non_executed_quantity_final = total_order_quantity - current_executed_quantity

non_executed_quantity_final

3000

In [32]:
final_df.to_csv("Long_Final_Executed_Data.csv", index=False)

# Final Extensions

In [33]:
final_df

,Timestamp,Order_ID,Instrument,Price,Units,Side
0,2023-05-11 15:00:01.141069,20.0,USD_CHF,0.89433,1000,SELL
1,2023-05-11 15:06:01.229906,24.0,USD_CHF,0.89443,1000,SELL
2,2023-05-11 15:12:01.337653,30.0,USD_CHF,0.89409,1000,SELL
3,2023-05-11 15:18:01.444667,38.0,USD_CHF,0.89409,1000,SELL
4,2023-05-11 15:24:01.544418,42.0,USD_CHF,0.89408,1000,SELL
...,...,...,...,...,...,...
22,2023-05-12 05:12:07.816411,373.0,USD_CHF,0.89157,1000,SELL
23,2023-05-12 05:18:08.046387,376.0,USD_CHF,0.89239,1000,SELL
24,2023-05-12 05:24:08.298965,379.0,USD_CHF,0.89284,1000,SELL
25,2023-05-12 05:30:08.499779,382.0,USD_CHF,0.89281,1000,SELL


In [34]:
current_executed_quantity

97000

In [35]:
non_executed_quantity_final

3000

In [36]:
if non_executed_quantity_final > 0:
    
    # Wait for 30 minutes after last execution window
    extension1_end_time = fourth_execution_end_time + dt.timedelta(minutes=30)
    while dt.datetime.now() < extension1_end_time:
        time.sleep(60)

    extension1_executed_trades = []
    new_order_quantity = int(non_executed_quantity_final * 0.5)

    # Check condition and execute trade
    if get_rate() <= final_average_execution_price:
        execute_order(new_order_quantity, instrument, "SELL", extension1_executed_trades)
        print("Traded 50% of the Non Executed Quantity in one go as Price <= Average Price")
        
    else:
        print("Trading did not take place as Price > Average Price.")
        
else:
    print("Trading did not take place as the entire quantity was executed in the previous four execution windows.")

Traded 50% of the Non Executed Quantity in one go as Price <= Average Price


In [37]:
# convert the list of prices into a DataFrame
extension1_df = pd.DataFrame(extension1_executed_trades)

extension1_df

,Timestamp,Order_ID,Instrument,Price,Units,Side
0,2023-05-12 06:30:17.079835,388.0,USD_CHF,0.89271,1500,SELL


In [38]:
if non_executed_quantity_final > 0:
    
    # Wait for 60 minutes after last execution window
    extension2_end_time = fourth_execution_end_time + dt.timedelta(minutes=60)
    while dt.datetime.now() < extension2_end_time:
        time.sleep(60)

    extension2_executed_trades = []

    # Check condition and execute trade
    if not extension1_df.empty:
        # If current Price is more than the Average Execution Price
        if get_rate() <= final_average_execution_price:
            # Buy remaining 50% of non-executed amount
            execute_order(new_order_quantity, instrument, "SELL", extension2_executed_trades)
            print("Traded remaining 50% of the Non Executed Quantity in one go as Price <= Average Price and Extention1 was executed.")
        # If If current Price is less than the Average Execution Price
        else:
            # Buy remaining 50% of non-executed amount
            execute_order(new_order_quantity, instrument, "SELL", extension2_executed_trades)
            print("Traded remaining 50% of the Non Executed Quantity in one go as Price > Average Price and Extention1 was executed.")
     
    else:
        # Check if current price is below average price, then start selling
        if get_rate() > final_average_execution_price:
            # Sell total executed units every one-minute (TWAP) whatever the price is until 20% of initial order is reached
            required_quantity = current_executed_quantity - (total_order_quantity * 0.2)
            executed_quantity = 0

            # Keep selling until executed_quantity < new_order_size
            while executed_quantity < required_quantity:
                execute_order(order_quantity, instrument, "BUY", extension2_executed_trades)
                executed_quantity += order_quantity
                time.sleep(60)
                print("Successful execution!")
            print("Order quantity reached below 20% of the initial order.")
            
        else:
            print ("Trading did not take place as Price < Average Price.")
            
else:
    print("Trading did not take place as the entire quantity was executed in the previous four execution windows.")

Traded remaining 50% of the Non Executed Quantity in one go as Price > Average Price and Extention1 was executed.


In [39]:
# convert the list of prices into a DataFrame
extension2_df = pd.DataFrame(extension2_executed_trades)

extension2_df

,Timestamp,Order_ID,Instrument,Price,Units,Side
0,2023-05-12 07:00:17.606450,393.0,USD_CHF,0.89332,1500,SELL


# Updating the Database

In [40]:
from pymongo import MongoClient

# MongoDb Setup
client = MongoClient('mongodb://localhost:27017/')
db = client["TWAP_Forex_Trading"]

In [41]:
collection_name = "Short_Trade_USDCHF"
collection = db[collection_name]
collection.insert_many(final_df.to_dict("records") + extension1_df.to_dict("records") + extension2_df.to_dict("records"))

# Creating csv File

In [44]:
# Create a dictionary with your data
data = {'Interval': ['Interval 1', 'Interval 2', 'Interval 3', 'Interval 4'],
        'Executed Quantity': [200000, 16000, 20000, 27000], 
        'Percentage of Executed Quantity': [100, 53.33, 100, 90],
        'Non Executed Quantity': [0, 14000, 0, 3000],
        'Percentage of Non Executed Quantity': [0, 46.67, 0, 10]}

# Create a Pandas DataFrame from the dictionary
df = pd.DataFrame(data)

# Add some sample data for the last three columns
df['Average Execution Price'] = [average_execution_price1, average_execution_price2, average_execution_price3, final_average_execution_price]

# Convert the DataFrame to a CSV file
df.to_csv('Final Project - Short - Execution Output Vectors.csv', index=False)